# **Importar librerias**

In [1]:
# Imports
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# **Cargar la base de datos**

In [2]:
# Load the English STSB dataset
stsb_dataset = load_dataset('stsb_multi_mt', 'en')
stsb_train = pd.DataFrame(stsb_dataset['train'])
stsb_test = pd.DataFrame(stsb_dataset['test'])

# Check loaded data
print(stsb_train.shape, stsb_test.shape)
stsb_test.head()

(5749, 3) (1379, 3)


,sentence1,sentence2,similarity_score
0,A girl is styling her hair.,A girl is brushing her hair.,2.5
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,3.6
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,5.0
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,4.2
4,A man is playing a harp.,A man is playing a keyboard.,1.5


# **Crear Funciones Útiles**

La primera función tiene como objetivo preprocesar textos mediante la lematización, convertir a minúsculas y eliminar números y palabras de parada. 
La segunda función toma dos columnas de incrustaciones de texto y devuelve la similitud del coseno entre las dos columnas a nivel de filas.

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import spacy

nlp = spacy.load("en_core_web_sm")

def text_processing(sentence):
    """
    Lemmatize, lowercase, remove numbers and stop words
    
    Args:
      sentence: The sentence we want to process.
    
    Returns:
      A list of processed words
    """
    sentence = [token.lemma_.lower()
                for token in nlp(sentence) 
                if token.is_alpha and not token.is_stop]
    
    return sentence


def cos_sim(sentence1_emb, sentence2_emb):
    """
    Cosine similarity between two columns of sentence embeddings
    
    Args:
      sentence1_emb: sentence1 embedding column
      sentence2_emb: sentence2 embedding column
    
    Returns:
      The row-wise cosine similarity between the two columns.
      For instance is sentence1_emb=[a,b,c] and sentence2_emb=[x,y,z]
      Then the result is [cosine_similarity(a,x), cosine_similarity(b,y), cosine_similarity(c,z)]
    """
    cos_sim = cosine_similarity(sentence1_emb, sentence2_emb)
    return np.diag(cos_sim)

# **Medir similaridad semántica con algoritmos clasicos sin contexto**

## Jaccard Similarity:

In [4]:
import textdistance

def jaccard_sim(row):
    # Text Processing
    sentence1 = text_processing(row['sentence1'])
    sentence2 = text_processing(row['sentence2'])
    
    # Jaccard similarity
    return textdistance.jaccard.normalized_similarity(sentence1, sentence2)


# Jaccard Similarity
stsb_test['Jaccard_score'] = stsb_test.progress_apply(jaccard_sim, axis=1)

100%|██████████| 1379/1379 [00:18<00:00, 72.62it/s]


## Bag of Words (BoW)

### Count Vectorizer

### TFIDF Vectorizer

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer(lowercase=True, stop_words='english')

# Train the model
X_train = pd.concat([stsb_train['sentence1'], stsb_train['sentence2']]).unique()
model.fit(X_train)

# Generate Embeddings on Test
sentence1_emb = model.transform(stsb_test['sentence1'])
sentence2_emb = model.transform(stsb_test['sentence2'])

# Cosine Similarity
stsb_test['TFIDF_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)

## Word Movers Distance (WMD)

In [7]:
pip install pot

   ---------------------------------------- 0.0/253.5 kB ? eta -:--:--
   --------- ------------------------------ 61.4/253.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 253.5/253.5 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import gensim.downloader as api

# Load the pre-trained model
model = api.load('fasttext-wiki-news-subwords-300')

def word_movers_distance(row):
    # Text Processing
    sentence1 = text_processing(row['sentence1'])
    sentence2 = text_processing(row['sentence2'])
    
    # Negative Word Movers Distance
    return -model.wmdistance(sentence1, sentence2)


# Negative Word Movers Distance
stsb_test['NegWMD_score'] = stsb_test.progress_apply(word_movers_distance, axis=1)

  0%|          | 0/1379 [00:00<?, ?it/s]c:\Users\diana\Miniconda3\envs\data\lib\site-packages\ot\backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())
100%|██████████| 1379/1379 [00:39<00:00, 34.64it/s]


# **Medir Similaridad Semántica con algoritmos modernos con contexto**

## Universal Sentence Encoder (USE)

In [9]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained model
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    # Control GPU memory usage
    tf.config.experimental.set_memory_growth(gpu, True)

module_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'
model = hub.load(module_url)

# Generate Embeddings
sentence1_emb = model(stsb_test['sentence1']).numpy()
sentence2_emb = model(stsb_test['sentence2']).numpy()

# Cosine Similarity
stsb_test['USE_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)

## Cross Encoder

In [10]:
from sentence_transformers import CrossEncoder

# Load the pre-trained model
model = CrossEncoder('cross-encoder/stsb-roberta-base')

sentence_pairs = []
for sentence1, sentence2 in zip(stsb_test['sentence1'], stsb_test['sentence2']):
    sentence_pairs.append([sentence1, sentence2])
    
stsb_test['SBERT CrossEncoder_score'] = model.predict(sentence_pairs, show_progress_bar=True)

c:\Users\diana\Miniconda3\envs\data\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\diana\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

## SBERT Bi-Encoder

In [11]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('stsb-mpnet-base-v2')

# Generate Embeddings
sentence1_emb = model.encode(stsb_test['sentence1'], show_progress_bar=True)
sentence2_emb = model.encode(stsb_test['sentence2'], show_progress_bar=True)

# Cosine Similarity
stsb_test['SBERT BiEncoder_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

## SimCSE

In [12]:
########## Supervised ##########
# Load the pre-trained model
model = SentenceTransformer('princeton-nlp/sup-simcse-roberta-large')

# Generate Embeddings
sentence1_emb = model.encode(stsb_test['sentence1'], show_progress_bar=True)
sentence2_emb = model.encode(stsb_test['sentence2'], show_progress_bar=True)

# Cosine Similarity
stsb_test['SimCSE Supervised_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)


########## Un-Supervised ##########
# Load the pre-trained model
model = SentenceTransformer('princeton-nlp/unsup-simcse-roberta-large')

# Generate Embeddings
sentence1_emb = model.encode(stsb_test['sentence1'], show_progress_bar=True)
sentence2_emb = model.encode(stsb_test['sentence2'], show_progress_bar=True)

# Cosine Similarity
stsb_test['SimCSE Unsupervised_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

## OpenAI

In [13]:
"""
import openai
import os
import pickle
openai.api_key = 'update_your_openai_API_key_here'

if os.path.exists('../data/nlp/davinci_emb.pkl'):
    print('Loading Davinci Embeddings')
    with open('../data/nlp/davinci_emb.pkl', 'rb') as f:
        davinci_emb = pickle.load(f)
else:
    print('Querying Davinci Embeddings')
    davinci_emb = {}
    engine='text-similarity-davinci-001'

    unique_sentences = list(set(stsb_test['sentence1'].values.tolist() + stsb_test['sentence2'].values.tolist()))
    for sentence in tqdm(unique_sentences):
        if sentence not in davinci_emb.keys():
            davinci_emb[sentence] = openai.Embedding.create(input = [sentence], 
                                                            engine=engine)['data'][0]['embedding']
    # Save embeddings to file      
    with open('../data/nlp/davinci_emb.pkl', 'wb') as f:
        pickle.dump(davinci_emb, f)

# Generate Embeddings
sentence1_emb = [davinci_emb[sentence] for sentence in stsb_test['sentence1']]
sentence2_emb = [davinci_emb[sentence] for sentence in stsb_test['sentence2']]

# Cosine Similarity
stsb_test['OpenAI Davinci_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)
"""

"\nimport openai\nimport os\nimport pickle\nopenai.api_key = 'update_your_openai_API_key_here'\n\nif os.path.exists('../data/nlp/davinci_emb.pkl'):\n    print('Loading Davinci Embeddings')\n    with open('../data/nlp/davinci_emb.pkl', 'rb') as f:\n        davinci_emb = pickle.load(f)\nelse:\n    print('Querying Davinci Embeddings')\n    davinci_emb = {}\n    engine='text-similarity-davinci-001'\n\n    unique_sentences = list(set(stsb_test['sentence1'].values.tolist() + stsb_test['sentence2'].values.tolist()))\n    for sentence in tqdm(unique_sentences):\n        if sentence not in davinci_emb.keys():\n            davinci_emb[sentence] = openai.Embedding.create(input = [sentence], \n                                                            engine=engine)['data'][0]['embedding']\n    # Save embeddings to file      \n    with open('../data/nlp/davinci_emb.pkl', 'wb') as f:\n        pickle.dump(davinci_emb, f)\n\n# Generate Embeddings\nsentence1_emb = [davinci_emb[sentence] for sentence 

# Evaluar resultados

In [17]:
score_cols = [col for col in stsb_test.columns if '_score' in col]

# Spearman Rank Correlation
spearman_rank_corr = stsb_test[score_cols].corr(method='spearman').iloc[1:, 0:1]*100
spearman_rank_corr.head(10)

,similarity_score
Jaccard_score,65.871317
TFIDF_cosine_score,61.420989
NegWMD_score,66.902825
USE_cosine_score,77.085989
SBERT CrossEncoder_score,90.172534
SBERT BiEncoder_cosine_score,88.572413
SimCSE Supervised_cosine_score,87.082275
SimCSE Unsupervised_cosine_score,82.784251


In [15]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

nrows = 4
ncols = 3
plot_array = np.arange(0, nrows*ncols).reshape(nrows, ncols)

subplot_titles = [f'{row.Index.split("_")[0]}: {row.similarity_score:.2f}' for row in spearman_rank_corr.itertuples()]
fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=subplot_titles)

for index, score in enumerate(spearman_rank_corr.index):
    row, col = np.argwhere(plot_array == index)[0]
    
    fig.add_trace(
        go.Scatter(
            x=stsb_test[score_cols[0]], 
            y=stsb_test[score],
            mode='markers',
        ),
        row=row+1, col=col+1
    )


fig.update_layout(height=700, width=1000, title_text='Spearman Rank Correlation (ρ × 100)', showlegend=False)
fig.show()

# **Referencias**

Leo, M. S. (2022, abril 25). Semantic textual similarity. Towards Data Science. https://towardsdatascience.com/semantic-textual-similarity-83b3ca4a840e

# **Cargar datos del Covid-19**

In [49]:
url = "C:\\Users\\diana\\OneDrive\\Escritorio\\Python\\CorpusTranslated.xlsx"
df=pd.read_excel(url)

df

,Query,Eng-Query,Old-intent,New-intent
0,bulos,hoaxes,bulo,bulo
1,tengo @sintomas,I have @symptoms,sintomas,sintomas
2,me duele el pecho,my chest hurts,sintomas,sintomas
3,De donde habeis sacado la @informacion?,Where did you get the @informacion?,fuentes_oficiales,documentos
4,Que tengo que hacer si tengo @sintomas?,What do I have to do if I have @symptoms?,sintomas_quehacer,sintomas
...,...,...,...,...
139243,Si es guapa,if she is pretty,transporte-comunidades,viajar
139244,Cuanto casos virus hoy,How many virus cases today,cifras,cifras
139245,Puedo llevar a mi mujer en @transporte a una c...,Can I take my wife in @transporte to an appoin...,libertad_circulacion_coche,viajar
139246,Se @cultura_deporteede trabajar sin medidas de...,Be @cultura_deporteede work without protection...,salir_casa_recomendaciones,salidas


In [33]:
df["New-intent"].unique()

array(['bulo', 'sintomas', 'documentos', 'cifras', 'contagio', 'salidas',
       'viajar', 'trabajo', 'tiendas', 'instrucciones', 'mascotas',
       'convivencia', 'salir_correrbici_paseo', 'ceremonias', 'educacion',
       'estado-alarma', 'alimentos', 'medicinas', 'peluqueria', nan,
       'ayuda', 'periodo_incubacion', 'recaida', 'no-gracias',
       'coronavirus', 'mascarillas', 'suministro', 'hospital',
       'otras-enfermedades', 'aislamiento_convivir', 'donar-sangre',
       'disculpas', 'insulto', 'iniciativas', 'estigma', 'ciberseguridad',
       'recuperacion', 'profesiones', 'areas_transmision', 'desescalada'],
      dtype=object)

In [31]:
df["New-intent"].value_counts()

New-intent
salidas                   20652
viajar                    19507
estado-alarma             13245
cifras                     9338
contagio                   8856
sintomas                   5337
tiendas                    5330
trabajo                    4421
mascarillas                3828
convivencia                3561
instrucciones              3149
medicinas                  2936
ayuda                      2741
documentos                 2733
alimentos                  2528
educacion                  2067
peluqueria                 2042
coronavirus                1736
salir_correrbici_paseo     1424
hospital                   1255
iniciativas                1241
disculpas                  1147
bulo                       1087
no-gracias                 1046
mascotas                    953
otras-enfermedades          872
ceremonias                  606
insulto                     530
suministro                  399
aislamiento_convivir        238
recaida                     1

In [32]:
# Revisión del porcentaje de datos nulos en cada una de las variables
df.isnull().sum()/len(df)*100

Query          0.000000
Eng-Query     10.173934
Old-intent     0.000000
New-intent    10.003734
dtype: float64

In [41]:
# Eliminar filas vacías
df = df.dropna()

In [47]:
df_prueba=df[df["New-intent"]=="salidas"]
df_prueba

,Query,Eng-Query,Old-intent,New-intent
7,Puedo hacer o recibir visitas?,Can I make or receive visits?,salir_recibir_visitas,salidas
8,limitaciones de libre circulacion de vehiculos.,Limitations of free circulation of vehicles.,salir_condiciones,salidas
10,transporte @cultura_deporteblico,transport @cultura_deporteblico,salir_transportepublico,salidas
11,Puedo @viajarl trabajo?,Can I @travel work?,salir_trabajar_permiso,salidas
18,condiciones para salir a la calle,conditions to go outside,salir_condiciones,salidas
...,...,...,...,...
139224,Mi familia me dice que no fie de los desconocidos,My family tells me not to trust strangers,salir_recibir_visitas,salidas
139226,Que sitios no se podrá visitar?,What sites can not be visited?,salir_recibir_visitas,salidas
139235,Puedo @viajar casa d mi familia?,Can I travel to my family's house?,salir_recibir_visitas,salidas
139241,Hola ya dos meses en casa esta bien quiero @...,"Hello, two months at home, it's fine, I want @...",salir_trabajar_permiso,salidas


In [54]:
# Crear un DataFrame vacío para almacenar los resultados
resultados = pd.DataFrame(columns=['Eng-Query1', 'Eng-Query2'])

# Iterar a través de las filas del DataFrame original
for i in range(len(df_prueba)):
    query1 = df_prueba.loc[i, 'Eng-Query']
    for j in range(i+1, len(df_prueba)):  # Comenzar desde i + 1 para evitar comparaciones repetidas
        query2 = df_prueba.loc[j, 'Eng-Query']
        resultados = resultados.append({'Eng-Query1': query1, 'Eng-Query2': query2}, ignore_index=True)

# El DataFrame 'resultados' ahora contiene todas las comparaciones sin repetir
print(resultados)

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types